In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Basic Data Quality checks
.missing values 
.duplicate rows
.basic data characteristics

In [3]:
def check_data_quality(df):
    """
    stores data quality metrics
    """
    quality_report = {
        'missing_values': df.isnull().sum().to_dict(),
        'duplicates': df.duplicated().sum(),
        'total_rows': len(df),
        'memory_usage': df.memory_usage().sum() / 1024**2  # in MB
    }
    return quality_report



# Standardize Data Types
Converting string dates to datetime objects
Identifying and converting numeric strings to actual numbers
Ensuring categorical variables are properly encoded
 

In [4]:
def standardize_datatypes(df):
    for column in df.columns:
        # Try converting string dates to datetime
        if df[column].dtype == 'object':
            try:
                df[column] = pd.to_datetime(df[column])
                print(f"Converted {column} to datetime")
            except ValueError:
                # Try converting to numeric if datetime fails
                try:
                    df[column] = pd.to_numeric(df[column].str.replace('$', '').str.replace(',', ''))
                    print(f"Converted {column} to numeric")
                except:
                    pass
    return df

# Handle Missing Values
Using median imputation for numeric columns
Applying mode imputation for categorical data
Maintaining the statistical properties of the dataset while filling gaps

In [12]:
from sklearn.impute import SimpleImputer

def handle_missing_values(df):
    # Handle numeric columns
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    if len(numeric_columns) > 0:
        num_imputer = SimpleImputer(strategy='median')
        df[numeric_columns] = num_imputer.fit_transform(df[numeric_columns])
    
    # Handle categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns
    if len(categorical_columns) > 0:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        df[categorical_columns] = cat_imputer.fit_transform(df[categorical_columns])
    
    return df

# Detect and Handle Outliers
You need domain knowledge to decide on “what” might actually be outliers.

Here's an approach using the Interquartile Range (IQR) method:

Calculate Interquartile Range (IQR) for numeric columns
Identify values beyond 1.5 * IQR from quartiles
Apply capping to extreme values rather than removing them
This preserves data while managing extreme values.
 

In [6]:
def remove_outliers(df):
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    outliers_removed = {}
    
    for column in numeric_columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers before removing
        
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)].shape[0]
        
        # Cap the values instead of removing them
        df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
        
        if outliers > 0:
            outliers_removed[column] = outliers
            
    return df, outliers_removed

# Validate the Results
 
After cleaning, we need to verify that our pipeline worked as expected:

Confirm no remaining missing values
Check for any remaining duplicates
Validate data integrity and consistency
Generate a comprehensive cleaning report

In [7]:
def validate_cleaning(df, original_shape, cleaning_report):
    validation_results = {
        'rows_remaining': len(df),
        'missing_values_remaining': df.isnull().sum().sum(),
        'duplicates_remaining': df.duplicated().sum(),
        'data_loss_percentage': (1 - len(df)/original_shape[0]) * 100
    }
    
    # Add validation results to the cleaning report
    cleaning_report['validation'] = validation_results
    return cleaning_report

# Finally, let's put it all together in a complete pipeline:

In [13]:
def automated_cleaning_pipeline(df):
    # Store original shape for reporting
    original_shape = df.shape
    
    # Initialize cleaning report
    cleaning_report = {}
    
    # Execute each step and collect metrics
    cleaning_report['initial_quality'] = check_data_quality(df)
    
    df = standardize_datatypes(df)
    df = handle_missing_values(df)
    df, outliers = remove_outliers(df)
    cleaning_report['outliers_removed'] = outliers
    
    # Validate and finalize report
    cleaning_report = validate_cleaning(df, original_shape, cleaning_report)
    
    return df, cleaning_report

In [14]:
automated_cleaning_pipeline(train_df)

(            userId  movieId  rating     timestamp
 0           5163.0  19779.5     4.0  1.518350e+09
 1         106343.0      5.0     4.5  1.206239e+09
 2         146790.0   5459.0     5.0  1.076216e+09
 3         106362.0  19779.5     2.0  1.423043e+09
 4           9041.0    366.0     3.0  8.333758e+08
 ...            ...      ...     ...           ...
 10000033  136395.0  19779.5     5.0  1.521235e+09
 10000034  140078.0    553.0     3.0  1.002581e+09
 10000035  154807.0  19779.5     4.0  1.227675e+09
 10000036   85805.0    327.0     4.0  1.479922e+09
 10000037  139457.0   1009.0     4.0  8.589849e+08
 
 [10000038 rows x 4 columns],
 {'initial_quality': {'missing_values': {'userId': 0,
    'movieId': 0,
    'rating': 0,
    'timestamp': 0},
   'duplicates': 0,
   'total_rows': 10000038,
   'memory_usage': 305.1770668029785},
  'outliers_removed': {'movieId': 2359041, 'rating': 468784},
  'validation': {'rows_remaining': 10000038,
   'missing_values_remaining': 0,
   'duplicates_rema

In [15]:
train_df.shape

(10000038, 4)